In [1]:
from azureml.core import Workspace, Datastore
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Dataset
from azure.ai.ml import MLClient
from azure.identity import InteractiveBrowserCredential

subscription_id = "0a94de80-6d3b-49f2-b3e9-ec5818862801"
resource_group = "buas-y2"
workspace_name = "Test"

# Log in using interactive Auth
auth = InteractiveLoginAuthentication()

# Connect to workspace
workspace = Workspace(subscription_id=subscription_id,
                      resource_group=resource_group,
                      workspace_name=workspace_name,
                      auth=auth,
                      )

# Connect to the ML Client
credential = InteractiveBrowserCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

In [2]:
# Check for existing compute targets
compute_targets = workspace.compute_targets

for name, ct in compute_targets.items():
    print(f'🖥️ {name}: {ct.type}, Provisioning state: {ct.provisioning_state}')

In [3]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DockerConfiguration

# Create a new environment
env = Environment(name="my-docker-environment")

# Create a base Docker image to use for the environment
env.docker.base_dockerfile = """
FROM ubuntu:18.04
RUN apt-get update && \
    apt-get install -y python3-pip && \
    pip3 install azureml-defaults
"""

# Create a new Conda dependencies object
conda_dep = CondaDependencies.create(python_version="3.8")

# Add some package dependencies
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("scikit-learn")

# Add the dependencies to the environment
env.python.conda_dependencies = conda_dep

# Specify a Docker-based environment
docker_config = DockerConfiguration(use_docker=True)

# Register the environment to re-use later
env.register(workspace)

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


{
    "assetId": "azureml://locations/westeurope/workspaces/db510a41-155d-4ea7-8569-34a7f7be6be0/environments/my-docker-environment/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "\nFROM ubuntu:18.04\nRUN apt-get update &&     apt-get install -y python3-pip &&     pip3 install azureml-defaults\n",
        "baseImage": null,
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inf

In [4]:
# List all available environments.
envs = ml_client.environments.list()
for env in envs:
    print(f"Environment name: {env.name}, latest version: {env.latest_version}")

Environment name: my-docker-environment, latest version: 1
Environment name: AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu, latest version: 10
